In [1]:
import pandas as pd
import numpy as np
import re

# 2017 Sensor Data Cleaning

In [2]:
# Read in the file. For iterations, start from cleaned file to skip initial datetime conversion
# When initially taken in, the ArrivalTime and DepartureTime had 2 noticeable formats
# Both 24HR and 12HR style
# Took a considerable amount of time to do the initial conversion 1hr+
# When saved to a csv again, does not retain dtype, but conversion time is short

#p_2017 = pd.read_csv('../Data/On-street_Car_Parking_Sensor_Data_-_2017.csv')

# Below csv had time converted, but also added a column and removed others on first save
# Because of this, some lines didn't need to be run in itterations, and have been commented out

p_2017 = pd.read_csv('../Data/cleaned/On-street_Car_Parking_Sensor_Data_-_2017_cleaned.csv')

In [3]:
# Put Python to work and convert Time columns to datetime dtype. Long runtime, avoid use
# Remove as many records as possible first, before running this

#p_2017['ArrivalTime'] = pd.to_datetime(p_2017['ArrivalTime'], infer_datetime_format=True)
#p_2017['DepartureTime'] = pd.to_datetime(p_2017['DepartureTime'], infer_datetime_format=True)

# Dropping columns to make the set more manageable
# DeviceId is redundant, sensors have been changed over the years
# StreetId doesn't provide useful information
# BetweenStreets also dropped for now, on merging will have lat/long info

#p_2017.drop(columns = ['DeviceId', 'StreetId', 'BetweenStreet1', 'BetweenStreet2'], inplace =True)

# New column if a sign is in effect at time of event
# Don't need to run now, was performed in first iteration of cleaning

#p_2017['Sign_in_effect'] = p_2017['sign'].notnull()

In [4]:
p_2017.head()

,ArrivalTime,DepartureTime,DurationSeconds,StreetMarker,Sign,Area,StreetName,Side Of Street,In Violation,Vehicle Present,Sign_in_effect
0,2017-04-11 07:24:35,2017-04-11 07:30:00,325,2477S,NaN,Spencer,BOURKE STREET,4,False,True,False
1,2017-07-03 06:41:20,2017-07-03 07:30:00,2920,3598N,NaN,Magistrates,LITTLE LONSDALE STREET,3,False,False,False
2,2017-04-07 10:48:38,2017-04-07 11:41:34,3176,12548N,2P TKT A M-F 7:30-18:30,Jolimont,WELLINGTON PARADE,3,False,True,True
3,2017-02-09 16:57:38,2017-02-09 17:11:04,806,13836N,2P TKT A M-SAT 7:30-18:30,Docklands,LITTLE DOCKLANDS DRIVE,3,False,True,True
4,2017-05-27 22:43:52,2017-05-27 22:47:28,216,3571S,NaN,Magistrates,LITTLE LONSDALE STREET,4,False,False,False


In [5]:
p_2017.dtypes

ArrivalTime        object
DepartureTime      object
DurationSeconds     int64
StreetMarker       object
Sign               object
Area               object
StreetName         object
Side Of Street      int64
In Violation         bool
Vehicle Present      bool
Sign_in_effect       bool
dtype: object

In [6]:
# Checking if there are any violations without signage
# Low number compared to dataset, so unconcerned about this at this point

p_2017[(p_2017['Sign'].isnull()) & (p_2017['In Violation'])]['In Violation'].count()

9

In [7]:
# Renaming columns in preferred convention

p_2017.rename(columns = {'ArrivalTime':'arrival_time',
                         'DepartureTime':'departure_time',
                         'DurationSeconds':'duration_seconds',
                         'StreetMarker':'street_marker',
                         'Sign':'sign',
                         'Area':'area',
                         'StreetName':'street_name',
                         'Side Of Street':'side_of_street',
                         'In Violation':'in_violation',
                         'Vehicle Present':'vehicle_present',
                         'Sign_in_effect':'sign_in_effect'},
                          inplace=True)

In [8]:
p_2017.columns

Index(['arrival_time', 'departure_time', 'duration_seconds', 'street_marker',
       'sign', 'area', 'street_name', 'side_of_street', 'in_violation',
       'vehicle_present', 'sign_in_effect'],
      dtype='object')

In [9]:
# Checking for nulls
# Nulls in sign just mean no sign was in effect during the parking event

p_2017.isnull().sum()

arrival_time               0
departure_time             0
duration_seconds           0
street_marker              0
sign                12364855
area                       0
street_name                0
side_of_street             0
in_violation               0
vehicle_present            0
sign_in_effect             0
dtype: int64

In [10]:
# There are few old entries in the Sign section, and in case 
# Sign will be used to merge onto the restrictions table
# outdated signs will be removed
# Also need to check if more signs are outdated when merging restrictions table 
# ..(Yes, yes they are) cannot use a merge onto restrictions, too many changes

p_2017 = p_2017[p_2017['sign'].str.contains('old', regex=False, na=False) == False]

In [11]:
# Duration in negative values or 0 are due to defective sensors, removing as corrupt data
# Found some durations greater than a day, impossible with way sensors log info, 
# removing as corrupt data

p_2017 = p_2017[p_2017['duration_seconds'] > 0]
p_2017 = p_2017[p_2017['duration_seconds'] < 86400]
p_2017.shape

(35850893, 11)

In [12]:
# Kensington is one single parking bay with low utilisation, removing from analysis

p_2017 = p_2017[p_2017['area'] != 'Kensington']

In [13]:
p_2017.shape

(35849853, 11)

In [14]:
# Sorting, Takes a while

p_2017 = p_2017.sort_values(by=['street_marker', 'arrival_time'], ignore_index=True)

In [15]:
# Producing a sorted list of sign options
# Used to check what to perform in the next function

sign_list = list(p_2017[p_2017['sign'].notnull()]['sign'].unique())
sign_series = pd.Series(sign_list)

# Uncomment to see the list of sign options

#list(sign_series.sort_values())

In [16]:
# Time to parse the sign out, need to find the allowable time limit at time of parking
# Initially wanted to use restriction data, but current signage does not match 2017 well enough

def sign_parse(signs):
    if signs == np.NAN:
        return np.NAN
    
# numP parking signs
    
    x = re.match(r"(^\S[^P]*)", signs)
    if x != None and x.group(1) == '1/4':    
        return 15*60
    elif x != None and x.group(1) == '1/2':
        return 30*60
    elif x != None and x.group(1) == '1':
        return 60*60
    elif x != None and x.group(1) == '2':
        return 120*60
    elif x != None and x.group(1) == '3':
        return 180*60
    elif x != None and x.group(1) == '4':
        return 240*60

# LZ num parking signs        
        
    y = re.search(r"((?<=LZ )\d\d)", signs)
    if y != None and y.group(0) == '30':    
        return 30*60
    elif y != None and y.group(0) == '15':
        return 15*60

# Pnum parking signs        
        
    z = re.search(r"(?<=P)\d+", signs)
    if z != None and z.group(0) == '15':    
        return 15*60
    if z != None and z.group(0) == '10':    
        return 10*60
    if z != None and z.group(0) == '5':    
        return 5*60

# P/num parking signs
        
    i = re.search(r"(?<=P/)\d+", signs)
    if i != None and i.group(0) == '15':    
        return 15*60
    if i != None and i.group(0) == '10':    
        return 10*60
    if i != None and i.group(0) == '5':    
        return 5*60
        
    if signs == 'Loading Zone 60mins':
        return 60*60
        
    if signs == 'L/Zone 30MINS 7.30 - 6.30PM':
        return 30*60
        
    if signs == 'P 5 Mon - Sat 7.30 - 19.30':
        return 5*60
    
    return np.NAN

In [17]:
# Creating a column using the parsing function

p_2017['duration_allowed'] = p_2017['sign'].\
apply(lambda x:np.NAN if x is np.NAN else sign_parse(x))

In [18]:
# Time to seperate out the other elements into Bool cols

p_2017['loading_zone'] = p_2017['sign'].str.contains('LZ|L/Zone|Loading', regex=True, na=False)
p_2017['paid_park'] = p_2017['sign'].str.contains('MTR|TKT|Meter|Ticket', regex=True, na=False)
p_2017['disabled_park'] = p_2017['sign'].str.contains('DIS', regex=True, na=False)

In [19]:
# Handling the sequential issues, need to remove records that..
# defaulted to midnight either on arrival or departure..
# as noted by the Data owner
# Using shift to compare a sorted record to the previous record
# Despite sensors not working on public holidays, does not appear to be an issue for this
# Removing all if these records as corrupt data due to sensing issues 
# (135k records for arrival midnight default)

# Removing the ARRIVAL midnight default behaviour (135k records for arrival midnight default)

p_2017 = p_2017[((p_2017['arrival_time'] != p_2017['departure_time'].\
         shift(periods=1, fill_value='0000-00-00 00:00:00')) &\
         (p_2017['arrival_time'].str.contains('00:00:00', regex=False)) &\
         (p_2017['street_marker'] == p_2017['street_marker'].\
         shift(periods=1, fill_value='NA'))) == False]

# Removing the DEPARTURE midnight default behaviour (40k records for departure midnight default)

p_2017 = p_2017[((p_2017['departure_time'] != p_2017['arrival_time'].\
         shift(periods=-1, fill_value='0000-00-00 00:00:00')) &\
         (p_2017['departure_time'].str.contains('00:00:00', regex=False)) &\
         (p_2017['street_marker'] == p_2017['street_marker'].\
         shift(periods=-1, fill_value='NA'))) == False]

p_2017.shape

(35663812, 15)

In [20]:
# Reset the index after deleting records

p_2017.reset_index(drop=True, inplace=True)

In [21]:
# Save to new cleaned file

p_2017.to_csv('../Data/cleaned/parking_2017_cleaned.csv',encoding='utf-8',index=False)

In [22]:
# Split into 2 and save - Vehicle present = True and Violation = True

p_2017_vehicle_true = p_2017[p_2017['vehicle_present']]
p_2017_vehicle_true.to_csv('../Data/cleaned/parking_2017_cleaned_vehicle_present.csv',encoding='utf-8',index=False)

p_2017_violation_true = p_2017[p_2017['in_violation']]
p_2017_violation_true.to_csv('../Data/cleaned/parking_2017_cleaned_violation.csv',encoding='utf-8',index=False)

# Parking Geometery Cleaning

In [23]:
# Read in the file

p_geom = pd.read_csv('../Data/On-street_Parking_Bays.csv')

In [24]:
# filtering table down to marker_id with values. Values with no marker_id fall outside the project scope

p_geom = p_geom[p_geom['marker_id'].notnull()]

In [25]:
# Removing geometry data, last edit and the rd_segment columns

p_geom.drop(columns = ['last_edit','rd_seg_id','rd_seg_dsc', 'meter_id'], inplace =True)
p_geom.head()

,the_geom,marker_id,bay_id
11,MULTIPOLYGON (((144.969314279727 -37.830072209...,8006E,14698.0
12,MULTIPOLYGON (((144.980791752861 -37.818535866...,15139S,16684.0
15,MULTIPOLYGON (((144.957969625279 -37.816797594...,2258N,13666.0
23,MULTIPOLYGON (((144.966449574995 -37.822830434...,8783W,19898.0
24,MULTIPOLYGON (((144.950359511361 -37.819413321...,13478E,14971.0


In [26]:
p_geom.shape

(6023, 3)

In [27]:
# Save to new cleaned file

p_geom.to_csv('../Data/cleaned/bay_geom_cleaned.csv',encoding='utf-8',index=False)

# Current Parking Sensor Cleaning

In [28]:
# Read in the file

p_sensor = pd.read_csv('../Data/On-street_Parking_Bay_Sensors.csv')

# Size of database fluctuates as sensors connect and disconnect

In [29]:
# Removing live status for initial analysis and location (redundant)

p_sensor.drop(columns = ['status','location'], inplace =True)
p_sensor.head()

,bay_id,st_marker_id,lat,lon
0,5544,12021W,-37.813137,144.982972
1,5559,12051W,-37.812308,144.983110
2,3820,C5494,-37.807235,144.954340
3,5396,11758E,-37.810055,144.975699
4,5484,11891W,-37.810443,144.978100


In [30]:
p_sensor.shape

(2520, 4)

In [31]:
# Save to new cleaned file

p_sensor.to_csv('../Data/cleaned/parking_sensors_cleaned.csv',encoding='utf-8',index=False)